In [ ]:
%load_ext autoreload
%autoreload 2
%run notebook_setup.py

In [ ]:
import sys
import os
from backtest import *
from pareto_store import *
from pure_funcs import calc_drawdowns
import pandas as pd
import ta 

In [ ]:
config = load_config('configs/template.json')
config['live']['approved_coins'] = 'BTC'
config['backtest']['exchanges'] = ['binance', 'bybit']
config['backtest']['combine_ohlcvs'] = True
config['backtest']['start_date'] = '2020-01-01'
config['live']['minimum_coin_age_days'] = 30
config = format_config(config)
res = await prepare_hlcvs_mss(config, 'combined' if config['backtest']['combine_ohlcvs'] else config['backtest']['exchanges'][0])
coins, hlcvs, mss, results_path, cache_dir, btc_usd_prices = res
#prices = btc_usd_prices
hlcv = hlcvs[:,0]

In [ ]:
plt.plot(hlcv[:,2])

In [ ]:
# Calculate ADX
df = pd.DataFrame(hlcv, columns=["high", "low", "close", "volume"])
df["adx_1"] = ta.trend.adx(df["high"], df["low"], df["close"], window=30)
df["adx_2"] = ta.trend.adx(df["high"], df["low"], df["close"], window=10)
df["adx_1"].fillna(method="bfill", inplace=True)
df["adx_2"].fillna(method="bfill", inplace=True)
adx_1 = df["adx_1"].values
adx_2 = df["adx_2"].values

In [ ]:
results = {(0.01632, 1.518, 51.54, 0.02986, 0.02407, 0.005983, 0.01054, 1.3, 0.7, 10): None}
eligible = {}

In [ ]:
len(eligible)

In [ ]:
def perform_backtest(hlcv, adx_1, adx_2, key):
    (
        initial_qty_pct,
        double_down_factor,
        wallet_exposure_limit,
        trailing_threshold_pct_profit,
        trailing_threshold_pct_loss,
        trailing_retracement_pct_profit,
        trailing_retracement_pct_loss,
        adx_scale_higher_width,
        adx_scale_lower_width,
        max_flips_per_cycle,
    ) = key
    fee_rate = 0.00055
    fills, equities = pbr.backtest_trailing_flip(
        hlcv,
        adx_1,
        adx_2,
        initial_qty_pct,
        double_down_factor,
        wallet_exposure_limit,
        trailing_threshold_pct_profit,
        trailing_retracement_pct_profit,
        trailing_threshold_pct_loss,
        trailing_retracement_pct_loss,
        fee_rate,
        adx_scale_higher_width,
        adx_scale_lower_width,
        int(max_flips_per_cycle),  # Ensure int
    )
    fdf = pd.DataFrame(fills, columns=["index", "pnl", "fee_paid", "balance", "qty", "price", "psize", "pprice", "upnl", "type"]).set_index('index')
    edf = pd.Series(equities)
    fdf.loc[:,'wallet_exposure'] = fdf.psize.abs() * fdf.pprice / fdf.balance
    fdf.loc[:,'equity'] = fdf.balance + fdf.upnl
    drawdowns = calc_drawdowns(edf)
    drawdown_worst = abs(drawdowns.min())
    gain = edf.iloc[-1] / edf.iloc[0]
    n_days = len(edf) / 60 / 24
    adg = gain ** (1 / n_days) - 1 if gain > 0.0 else gain
    return {'drawdown_worst': drawdown_worst, 'gain': gain}

In [ ]:
begin_harmony_search = 20
harmony_memory_size = 20
limit_drawdown_worst = 0.50
n_iters = 40000
max_flips_upper_bound = 6
max_flips_lower_bound = 1

for k in results:
    if results[k] is None:
        results[k] = perform_backtest(hlcv, adx_1, adx_2, k)
        print('backtested', k, results[k])
        
eligible = {k: v for k, v in results.items() if v and v['drawdown_worst'] < limit_drawdown_worst}
for z in range(n_iters):
    if len(eligible) > begin_harmony_search:
        mode = 'harmony_search'
        resfs = sorted(eligible.items(), key=lambda x: x[1]['gain'])[-harmony_memory_size:]
        candidate = []
        for i in range(len(resfs[0][0])):
            vals = [resfs[k][0][i] for k in range(len(resfs))]
            candidate.append((np.random.choice(vals) + np.random.uniform(-0.000001, 0.000001)) * np.random.uniform(0.9, 1.1))
        # Clamp max_flips_per_cycle to wanted bounds
        candidate[-1] = int(np.clip(round(candidate[-1]), max_flips_lower_bound, max_flips_upper_bound))
    else:
        mode = 'random'
        initial_qty_pct = np.random.uniform(0.01, 1.0)
        double_down_factor = np.random.uniform(1.1, 3.0)
        wallet_exposure_limit = np.random.uniform(10.0, 100.0)
        trailing_threshold_pct_profit = np.random.uniform(0.01, 0.03)
        trailing_threshold_pct_loss = np.random.uniform(0.01, 0.03)
        trailing_retracement_pct_profit = np.random.uniform(0.00001, 0.01)
        trailing_retracement_pct_loss = np.random.uniform(0.00001, 0.01)
        adx_scale_higher_width = np.random.uniform(0.0, 3.0)
        adx_scale_lower_width = np.random.uniform(0.0, 3.0)
        max_flips_per_cycle = int(np.random.randint(max_flips_lower_bound, max_flips_upper_bound + 1))

        candidate = [
            initial_qty_pct,
            double_down_factor,
            wallet_exposure_limit,
            trailing_threshold_pct_profit,
            trailing_threshold_pct_loss,
            trailing_retracement_pct_profit,
            trailing_retracement_pct_loss,
            adx_scale_higher_width,
            adx_scale_lower_width,
            max_flips_per_cycle,
        ]
        
    key = (
        pbr.round_dynamic(candidate[0], 4),
        pbr.round_dynamic(candidate[1], 4),
        pbr.round_dynamic(candidate[2], 4),
        pbr.round_dynamic(candidate[3], 4),
        pbr.round_dynamic(candidate[4], 4),
        pbr.round_dynamic(candidate[5], 4),
        pbr.round_dynamic(candidate[6], 4),
        pbr.round_dynamic(candidate[7], 4),
        pbr.round_dynamic(candidate[8], 4),
        int(candidate[9]),  # Ensure int
    )
    if key in results:
        continue
    result = perform_backtest(hlcv, adx_1, adx_2, key)
    results[key] = result
    if result['drawdown_worst'] < limit_drawdown_worst:
        eligible[key] = result
        print(z, len(eligible), mode, key, eligible[key])
        

In [ ]:
resf = {k: v for k, v in results.items() if v['drawdown_worst'] < limit_drawdown_worst}
resfs = sorted(resf.items(), key=lambda x: x[1]['gain'])[-harmony_memory_size:]
resfs

In [ ]:
if not resfs:
    print("No eligible candidates found.")
    # Skip further processing
    raise ValueError("No eligible candidates found. Adjust parameters or increase iterations.")

candidate = resfs[-1][0]
fee_rate = 0.00055
nitial_qty_pct, double_down_factor, wallet_exposure_limit, trailing_threshold_pct_profit, trailing_threshold_pct_loss, trailing_retracement_pct_profit, trailing_retracement_pct_loss, adx_scale_higher_width, adx_scale_lower_width, max_flips_per_cycle = candidate
fills, equities = pbr.backtest_trailing_flip(
    hlcv,
    adx_1,
    adx_2,
    initial_qty_pct,
    double_down_factor,
    wallet_exposure_limit,
    trailing_threshold_pct_profit,
    trailing_retracement_pct_profit,
    trailing_threshold_pct_loss,
    trailing_retracement_pct_loss,
    fee_rate,
    adx_scale_higher_width,
    adx_scale_lower_width,
    int(max_flips_per_cycle),  # Ensure int
)
fdf = pd.DataFrame(fills, columns=["index", "pnl", "fee_paid", "balance", "qty", "price", "psize", "pprice", "upnl", "type"]).set_index('index')
edf = pd.Series(equities)
fdf.loc[:,'wallet_exposure'] = fdf.psize.abs() * fdf.pprice / fdf.balance
fdf.loc[:,'equity'] = fdf.balance + fdf.upnl
drawdowns = calc_drawdowns(edf)
drawdown_worst = abs(drawdowns.min())
gain = edf.iloc[-1] / edf.iloc[0]
n_days = len(edf) / 60 / 24
adg = gain ** (1 / n_days) - 1
print('drawdown_worst', drawdown_worst)
print('n_days', n_days)
print('n fills', len(fdf))
print('adg', adg)
print('gain', gain)
print('WE max', fdf.wallet_exposure.max())

param_names = [
    "initial_qty_pct",
    "double_down_factor",
    "wallet_exposure_limit",
    "trailing_threshold_pct_profit",
    "trailing_threshold_pct_loss",
    "trailing_retracement_pct_profit",
    "trailing_retracement_pct_loss",
    "adx_scale_higher_width",
    "adx_scale_lower_width",
    "max_flips_per_cycle",
]
print()
for name, value in zip(param_names, candidate):
    print(f"{name}: {value}")

In [ ]:
edf.plot(logy=True)

In [ ]:
edf.plot(logy=False)

In [ ]:
cost_pct = (fdf.qty.abs() * fdf.price) / fdf.balance
cost_pct.groupby(cost_pct.index // 1440).sum().mean()

In [ ]:
fdf.iloc[:60]

In [ ]:
exchange_min_order = 5

candidate = resfs[-1][0]
fee_rate = 0.00055
initial_qty_pct, double_down_factor, wallet_exposure_limit, trailing_threshold_pct_profit, trailing_threshold_pct_loss, trailing_retracement_pct_profit, trailing_retracement_pct_loss, adx_scale_higher_width, adx_scale_lower_width, max_flips_per_cycle = candidate

# Convert wallet_exposure_limit from percentage to decimal
wallet_exposure_limit /= 100 

# Calculate the total required balance for max flips
total_order_size = 0
for i in range(max_flips_per_cycle):
    total_order_size += exchange_min_order * (double_down_factor ** i)

# Adjust for wallet exposure limit
required_balance = total_order_size / wallet_exposure_limit

print(f"Minimum starting balance needed: {required_balance:.2f}")

In [ ]:
fdf.sort_values('wallet_exposure')

In [ ]:
i = 607126
n = 7000
fdfc = fdf.loc[i-n:i+n]
buys = fdfc[fdfc.qty > 0.0]
sells = fdfc[fdfc.qty < 0.0]
pd.DataFrame(hlcv[:,2]).loc[i-n:i+n].plot()
buys.price.plot(style='bo')
sells.price.plot(style='ro')

In [ ]:
fdfc

In [ ]:
edf.loc[fdfc.index[0]:fdfc.index[-1]].plot()

In [ ]:
fdf.sort_values('wallet_exposure').iloc[-10:]